## Import LSV from .mpr and calculate several J/V characteristics
Import the .mpr files directly. It will import all columns.

Generate a dataframe containing 2-electrode saturation photocurrent, zero bias photocurrent, and potential for 5 mA/cm2 current density for each file.

In [ ]:
import eclabfiles as ecf
import pandas as pd
import os
import os.path
import plotly.express as px

In [ ]:
# Need to collect all files and iterate through them, making a list of their names for later reference

# change directory_in_str to match where the .mpr files are
directory_in_str = '.'
filename_list = []
prefix_list = []

directory = os.fsencode(directory_in_str)

# generate list of full names and of prefix, i.e. electrode names, for example, WB832-5.
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".mpr"):
        filename_list.append(filename)
        #print(format(filename))
        name,ext = os.path.splitext(filename)
        #print(name)
        # add the filename to a list
        prefix_list.append(name[:7])
        #print(name[:7])
        continue
     else:
         continue
            
print(filename_list)
print(prefix_list)

In [ ]:
# initialize a data frame to put the results in and test concat
data = [['',0, 0, 0]]
summary_df = pd.DataFrame(columns=['Electrode','Jsat (mA/cm2)','J (0 bias)', 'V (5mA/cm2)'])
#summary_df = pd.concat([pd.DataFrame([['', 0, 0, 0]], columns = summary_df.columns), summary_df], ignore_index = True)
#print('summary df: \n', summary_df)

In [ ]:
# df2 will hold each file's results prior to concatenating it with the summary df
df2 = pd.DataFrame([['W', 11, 12, 13]], columns = summary_df.columns)
df2['Electrode'] = 'WW'
print(df2)
pd.concat([summary_df, df2])

In [ ]:
# organize the surface area in an excel file in the same folder, titled 'SA.xlsx'
# row 1: headings
# column 1: electrode name. column 2: surface area in cm2.

df_SA = pd.read_excel('SA.xlsx')
print(df_SA)

In [ ]:
# define the functions to use on each file
# first, one to edit the column names for consistency
def column_edit(df):
    # drop the columns you don't want
    df.drop(['time', '(Q-Qo)', 'I Range', 'mode', 'ox/red', 'error', 'control changes', 'uts'], axis=1, inplace=True)
    # rename the columns you do want
    df.rename(columns={'control_V': 'Control (V)', 'Ewe': 'Ewe (V vs. CE)', '<I>': 'I (mA)'}, inplace=True)
    #print(df.head())

In [ ]:
# second, a function to determine the surface area and calculate the current density
def J_calc(df, mpr_name):
    #print(mpr_name)
    #print(df_SA)
    # Now, search df_SA for mpr_name, and return the surface area in that same row.
    name_index = df_SA.index[df_SA['Electrode'] == mpr_name]
    #print('the row is: \n', df_SA.iloc[name_index])
    #print('printing the value in the row with the right name: ', df_SA.iloc[name_index]['surface area (cm2)'].iloc[0])
    elec_SA = df_SA.iloc[name_index]['surface area (cm2)']
    #print('the surface area is: ', elec_SA.iloc[0])
    current_density = df['I (mA)'].div(elec_SA.iloc[0])
    df['J (mA/cm2)'] = current_density
    #print('here is the dataframe with current density calculated: \n', df.head())

In [ ]:
# Third, calculate the saturation current density and return it to the calling function
def Jsat_calc(df):
    mindex = df.index[df['Ewe (V vs. CE)'] == df['Ewe (V vs. CE)'].min()]
    sat_photocurrent = df.iloc[mindex]['J (mA/cm2)']
    #print('The saturation photocurrent (mA/cm2) at -1 V vs. CE is: ', sat_photocurrent.iloc[0])
    return sat_photocurrent.iloc[0]

In [ ]:
# Fourth, calculate the photocurrent at zero bias for each LSV
# Bias is never exactly zero: goes from -4.9314409E-004 V to 4.9910863E-004 V.
# Can average photocurrent at these two points.
# find index by looking for voltages greater than -5E-4 and less than -4E-4
#print(df[df['Control (V)'] > -5E-4 & df['Control (V)'] < -4E-4])

def zero_bias(df):
    partial_df = df[df['Control (V)']>-5E-4]
    full_df = partial_df[partial_df['Control (V)'] < -4E-4]
    zerodex = full_df.index
    zero_bias_index = zerodex[0]
    #print(zero_bias_index)
    #print(full_df)
    # Now just average the current density at the zero bias index and the 
    Jsc = (df.iloc[zero_bias_index]['J (mA/cm2)'] + df.iloc[zero_bias_index + 1]['J (mA/cm2)'])/2
    #print('The short circuit current density (mA/cm2) is: ', Jsc)
    return Jsc

In [ ]:
# Fifth, calculate the lowest potential generating 5 mA/cm2 photocurrent.
#First, filter the ones that are at least 5 mA, then find the most positive bias which is applied within that set.
def five_mA_pot(df):
    first_df = df[df['J (mA/cm2)']<-5]
    #print(first_df)
    second_df = first_df[first_df['Control (V)'] == first_df['Control (V)'].max()]
    #print(second_df)
    # The type of the single row df second_df is a Pandas series:
    #print(type(second_df['Control (V)']))
    # provide the value of the series in the Control (V) column.
    five_V = second_df['Control (V)'].iloc[0]
    #print('The maximum potential (in V vs. CE) giving at least 5 mA/cm2 is: ' , five_V)
    return five_V

In [ ]:
# read each mpr file into a data frame
# edit columns and headings
# add current density column
# calculate saturation photocurrent and add to summary_dataframe

i = 0
for i in range(len(filename_list)):
   # print(filename_list[i])
    df2 = pd.DataFrame([['', 0, 0, 0]], columns = ['Electrode', 'Jsat (mA/cm2)', 'J (0 bias)', 'V (5mA/cm2)'])
    #print('prefix', prefix_list[i])
    df = ecf.to_df(filename_list[i])     
    column_edit(df)                      
    #print('file prefix is', prefix_list[i])
    df2['Electrode'] = prefix_list[i]
    print(prefix_list[i])
    J_calc(df, prefix_list[i])           
    Jsat_val = Jsat_calc(df)              
    df2['Jsat (mA/cm2)'] = Jsat_val
    zero_bias_val = zero_bias(df)                        
    df2['J (0 bias)'] = zero_bias_val
    five_mA_val = five_mA_pot(df) 
    df2['V (5mA/cm2)'] = five_mA_val
    print('df2\n', df2)
    summary_df = pd.concat([summary_df, df2])
print('summary_df\n', summary_df)

In [ ]:
# example from plotly
#ex_df = px.data.tips()
#fig = px.histogram(ex_df, x="total_bill", nbins=10)
#fig.show()
#print(ex_df)

# plot each variable
cols_y = list(summary_df.columns)[1:]
print(cols_y[1])
for y in cols_y:
    fig_ = px.histogram(summary_df[y], x = y, nbins = 3)
    fig_.show()
#fig_ = px.histogram(summary_df, x = 'Jsat (mA/cm2)', nbins = 5)
#fig_.show()
# plot just some variables
#cols_y = (df_all['Area_H2'], df_all['Area_O2'], df_all['I/mA']) #df_all['Area_O2XO'], df_all['Area_H2XO'], )
#for y in cols_y:
#    fig_ = px.scatter(df_all, x='timestamp', y=y)
#    fig_.show()
    
#print("Area_H2 and Area_O2")
#fig_ = px.scatter(summary_df, x='timestamp', y=['Area_O2', 'H2_ppm', 'Ewe/V'], log_y=True)
#fig_ = px.scatter(df_all, x='timestamp', y='Area_H2')
#fig_.show()